<a href="https://colab.research.google.com/github/robertzengcn/ai-finetuing/blob/master/small_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%pip install torch torchtext transformers sentencepiece pandas tqdm datasets

In [2]:
from datasets import load_dataset,DatasetDict,Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import os
import time

/home/robertzeng/anaconda3/envs/ai-finetuning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
data_sample=load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [6]:
updatad_data=[{'Name':item['Name'],'Symtoms':item['Symptoms']} for item in data_sample['train']]




In [7]:
df=pd.DataFrame(updatad_data)

In [8]:
df.head(5)

,Name,Symtoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [9]:
#just extract the symptoms
df['Symtoms']=df['Symtoms'].apply(lambda x: ', '.join(x.split(',')))

In [10]:
df['Symtoms']

0      Palpitations,  Sweating,  Trembling,  Shortnes...
1             Hoarseness,  Vocal Changes,  Vocal Fatigue
2      Short stature,  Gonadal dysgenesis,  Webbed ne...
3      Absence or undescended testicle(s),  empty scr...
4      Nausea,  vomiting,  abdominal pain,  General m...
                             ...                        
395    Severe abdominal or back pain,  blood in urine...
396    Fragile bones,  loss of height over time,  bac...
397    Joint pain,  stiffness,  swelling,  fatigue,  ...
398    Frequent urination,  Increased thirst,  Weight...
399    Fatigue,  Increased hunger,  Slow healing of w...
Name: Symtoms, Length: 400, dtype: object

In [11]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    try:
        device = torch.device("mps")    
    except Exception:
        device = torch.device("cpu")   

In [14]:
device

device(type='cuda')

In [15]:
tokenizer=GPT2Tokenizer.from_pretrained("distilgpt2")
model=GPT2LMHeadModel.from_pretrained("distilgpt2").to(device)

In [16]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
BATCH_SIZE = 8

In [18]:
df.describe()

,Name,Symtoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [19]:
# Dataset Prep
class LanguageDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.labels=df.columns
        self.data=df.to_dict(orient='records')
        self.tokenizer=tokenizer
        x=self.fittest_max_length(df)
        self.nax_length=x
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        x=self.data[idx][self.labels[0]]
        y=self.data[idx][self.labels[1]]
        text=f"{x} | {y}" 
        tokens=self.tokenizer.encode_plus(text,return_tensors='pt',max_length=128,padding='max_length')
        return tokens
    def fittest_max_length(self,df):
        max_length=max(len(max(df[self.labels[0]],key=len)),len(max(df[self.labels[1]],key=len)))
        x=2
        while x<max_length:x=x*2
        return x
            
        # self.df=df
        # self.tokenizer=tokenizer
        # self.max_length=max_length

In [20]:
data_sample=LanguageDataset(df,tokenizer)

In [21]:
data_sample

In [22]:
train_size=int(0.8*len(data_sample))
val_size=len(data_sample)-train_size
train_data,val_data=random_split(data_sample,[train_size,val_size])

In [23]:
#load train data
train_loader=DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
val_loader=DataLoader(val_data,batch_size=BATCH_SIZE)

In [24]:
num_epochs=8

In [25]:
batch_size=BATCH_SIZE
model_name='distilgpt2'
gpu=0

In [26]:
criterion=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer=optim.AdamW(model.parameters(),lr=5e-4)
tokenizer.pad_token=tokenizer.eos_token

In [27]:
results=pd.DataFrame(columns=['epoch','transformer','batch_size','gpu','training_loss','validation_loss','epoch_duration_sec'])

In [30]:

#train loop
for epoch in range(num_epochs):
    start_time=time.time()
    model.train()
    epoch_training_loss=0
    train_iterator=tqdm(train_loader,desc=f"Training epoch {epoch+1}/{num_epochs} Batch size: {batch_size},Transformer:{model_name}")
    for batch in train_iterator:
       optimizer.zero_grad()
       inputs=batch['input_ids'].squeeze(1).to(device)
       targets=inputs.clone()
       outputs=model(input_ids=inputs,labels=targets)
       loss=outputs.loss
       loss.backward()
       optimizer.step()
       train_iterator.set_postfix({'Training Loss':loss.item()})
       epoch_training_loss+=loss.item()
    #    train_iterator.set_postfix(loss=loss.item())
    avg_epoch_training_loss=epoch_training_loss/len(train_iterator)
    
    #validation
    model.eval()
    epoch_validation_loss=0
    total_loss=0
    valid_iterator=tqdm(val_loader,desc=f"Validating epoch {epoch+1}/{num_epochs} ")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs=batch['input_ids'].squeeze(1).to(device)
            targets=inputs.clone()
            outputs=model(input_ids=inputs,labels=targets)
            loss=outputs.loss
            total_loss+=loss
            valid_iterator.set_postfix({'Validation loss':loss.item()})
            epoch_validation_loss+=loss.item()
        avg_epoch_training_loss=epoch_validation_loss/len(valid_iterator)  
        
        end_time=time.time()
        epoch_duration_sec=end_time-start_time
        
        new_row={
            'transformer':model_name,
            'batch_size':batch_size,
            'gpu':gpu,
            'epoch':epoch+1,
            'training_loss':avg_epoch_training_loss,
            'validation_loss':avg_epoch_training_loss,
            }
        # results=results.append(new_row,ignore_index=True)
        results.loc[len(results)]=new_row
        print(f"Epoch:{epoch+1}, Validation Loss:{total_loss/len(val_loader)}")
          

Training epoch 1/8 Batch size: 8,Transformer:distilgpt2: 100%|██████████| 40/40 [00:03<00:00, 13.16it/s, Training Loss=0.574]
Validating epoch 1/8 : 100%|██████████| 10/10 [00:00<00:00, 52.38it/s, Validation loss=0.683]


Epoch:1, Validation Loss:0.6035404801368713


Training epoch 2/8 Batch size: 8,Transformer:distilgpt2: 100%|██████████| 40/40 [00:02<00:00, 14.02it/s, Training Loss=0.666]
Validating epoch 2/8 : 100%|██████████| 10/10 [00:00<00:00, 53.31it/s, Validation loss=0.724]


Epoch:2, Validation Loss:0.6212345957756042


Training epoch 3/8 Batch size: 8,Transformer:distilgpt2: 100%|██████████| 40/40 [00:02<00:00, 13.89it/s, Training Loss=0.446]
Validating epoch 3/8 : 100%|██████████| 10/10 [00:00<00:00, 50.87it/s, Validation loss=0.772]


...

Epoch:6, Validation Loss:0.7815186977386475


Training epoch 7/8 Batch size: 8,Transformer:distilgpt2: 100%|██████████| 40/40 [00:02<00:00, 13.63it/s, Training Loss=0.129] 
Validating epoch 7/8 : 100%|██████████| 10/10 [00:00<00:00, 51.85it/s, Validation loss=0.928]


Epoch:7, Validation Loss:0.8004545569419861


Training epoch 8/8 Batch size: 8,Transformer:distilgpt2: 100%|██████████| 40/40 [00:02<00:00, 13.43it/s, Training Loss=0.115] 
Validating epoch 8/8 : 100%|██████████| 10/10 [00:00<00:00, 52.90it/s, Validation loss=0.983]

Epoch:8, Validation Loss:0.840096652507782


In [31]:
!nvidia-smi

Fri Jul  4 15:26:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.64.01              Driver Version: 576.80         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   34C    P8             10W /  180W |    4076MiB /  16311MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [32]:
input_str="Kindney Failure"

In [33]:
input_ids=tokenizer.encode(input_str,return_tensors='pt').to(device)

In [34]:
input_ids

tensor([[35854,  1681, 25743]], device='cuda:0')

In [35]:
output=model.generate(
                      input_ids,
                      max_length=20,
                      num_return_sequences=1,
                      do_sample=True,
                      top_k=8,
                      top_p=0.95,
                      temperature=0.5,
                      repetition_penalty=1.2,
                     
                      )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [36]:
output

tensor([[35854,  1681, 25743,   930, 36400,   839, 18922,  5072,    11,   220,
         11711, 21545,    11,   220, 18787, 50256]], device='cuda:0')

In [37]:
decoded_output=tokenizer.decode(output[0],skip_special_tokens=True)

In [38]:
decoded_output

'Kindney Failure | Decreased urine output,  fluid retention,  fatigue'

In [ ]:
torch.save(model,'SmallDiseaseLM')

In [ ]:
#torch.save(model,'drive/My Drive/SmallDiseaseLM.pt')